In [2]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('pickle/data_dtm.pkl')
data

,aa,aal,aam,aba,aback,abacus,abalone,abandon,abandoned,abandonment,...,zoonosis,zoonotic,zorro,zoster,zowie,zucchini,zwanziger,zygomatic,αcoronavirus,βcoronavirus
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,55527,55528,55529,55530,55531,55532,55533,55534,55535,55536
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aam,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aback,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [11]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [12]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.025*"said" + 0.010*"coronavirus" + 0.010*"million" + 0.008*"company" + 0.008*"new" + 0.007*"year" + 0.007*"pandemic" + 0.007*"billion" + 0.006*"economy" + 0.006*"economic"'),
 (1,
  '0.018*"said" + 0.012*"coronavirus" + 0.012*"people" + 0.007*"time" + 0.007*"health" + 0.007*"new" + 0.005*"day" + 0.005*"newspaper" + 0.005*"just" + 0.005*"dont"')]